In [1]:
import requests
import json
import pprint
import pandas as pd
import pycurl
from itertools import chain
import numpy as np

pp = pprint.PrettyPrinter()

In [2]:
# This is an example of one pull from the balldontlie.io API for a player searched by name
# You can see that lebron james was added as a search term at the end, and we can replicate this with other players
# The goal being to isolate the id's to use later for performance statistics
# Next we will be creating a for loop to get all of the players from our salaries list

In [3]:
# As you can see in this example, any capitalization does not matter for the player search
james = requests.get("https://www.balldontlie.io/api/v1/players/?search=LEbron jaMes")
james.json()

{'data': [{'id': 237,
   'first_name': 'LeBron',
   'height_feet': 6,
   'height_inches': 8,
   'last_name': 'James',
   'position': 'F',
   'team': {'id': 14,
    'abbreviation': 'LAL',
    'city': 'Los Angeles',
    'conference': 'West',
    'division': 'Pacific',
    'full_name': 'Los Angeles Lakers',
    'name': 'Lakers'},
   'weight_pounds': 250}],
 'meta': {'total_pages': 1,
  'current_page': 1,
  'next_page': None,
  'per_page': 25,
  'total_count': 1}}

In [4]:
# We take the file from BDLio and convert it to a dictionary so we car parse the ID
james_dict = json.loads(james.text)
print(james_dict)
type(james_dict)

{'data': [{'id': 237, 'first_name': 'LeBron', 'height_feet': 6, 'height_inches': 8, 'last_name': 'James', 'position': 'F', 'team': {'id': 14, 'abbreviation': 'LAL', 'city': 'Los Angeles', 'conference': 'West', 'division': 'Pacific', 'full_name': 'Los Angeles Lakers', 'name': 'Lakers'}, 'weight_pounds': 250}], 'meta': {'total_pages': 1, 'current_page': 1, 'next_page': None, 'per_page': 25, 'total_count': 1}}


dict

In [5]:
james_id = james_dict['data'][0]["id"]

In [6]:
%store -r player_names_list
player_names_list

['Stephen Curry',
 'John Wall',
 'Russell Westbrook',
 'James Harden',
 'Damian Lillard',
 'LeBron James',
 'Kevin Durant',
 'Giannis Antetokounmpo',
 'Paul George',
 'Kawhi Leonard',
 'Klay Thompson',
 'Jimmy Butler',
 'Tobias Harris',
 'Khris Middleton',
 'Anthony Davis',
 'Rudy Gobert',
 'Kyrie Irving',
 'Bradley Beal',
 'Kristaps Porzingis',
 'Devin Booker']

In [7]:
url = "https://www.balldontlie.io/api/v1/players/?search="
player_id_list = []

for name in player_names_list:
    player_url = url + f"{name}"
    player_get = requests.get(player_url)
    player_get.json()
    player_dict = json.loads(player_get.text)
    player_id_list.append(player_dict['data'][0]["id"])
print(player_id_list)
%store player_id_list

[115, 1014, 472, 192, 278, 237, 140, 15, 172, 274, 443, 79, 200, 315, 117, 176, 228, 37, 378, 57]
Stored 'player_id_list' (list)


In [8]:
averages_url = "https://www.balldontlie.io/api/v1/season_averages?player_ids[]="
averages_list = []
for ids in player_id_list:
    averages_new = averages_url + f"{str(ids)}"
    averages_get = requests.get(averages_new)
    averages_get.json()
    averages_dict = json.loads(averages_get.text)
    averages_list.append(averages_dict['data'])
    if ids == 57:
        break
averages_list
  

[[{'games_played': 15,
   'player_id': 115,
   'season': 2021,
   'min': '32:35',
   'fgm': 8.87,
   'fga': 19.53,
   'fg3m': 5.27,
   'fg3a': 12.87,
   'ftm': 4.67,
   'fta': 4.87,
   'oreb': 0.67,
   'dreb': 5.53,
   'reb': 6.2,
   'ast': 6.47,
   'stl': 1.73,
   'blk': 0.6,
   'turnover': 3.27,
   'pf': 1.6,
   'pts': 27.67,
   'fg_pct': 0.454,
   'fg3_pct': 0.409,
   'ft_pct': 0.959}],
 [],
 [{'games_played': 15,
   'player_id': 472,
   'season': 2021,
   'min': '35:28',
   'fgm': 7.4,
   'fga': 17.33,
   'fg3m': 1.27,
   'fg3a': 4.33,
   'ftm': 3.33,
   'fta': 4.87,
   'oreb': 1.33,
   'dreb': 7.4,
   'reb': 8.73,
   'ast': 8.33,
   'stl': 1.4,
   'blk': 0.2,
   'turnover': 5.33,
   'pf': 3.2,
   'pts': 19.4,
   'fg_pct': 0.427,
   'fg3_pct': 0.292,
   'ft_pct': 0.685}],
 [{'games_played': 15,
   'player_id': 192,
   'season': 2021,
   'min': '34:11',
   'fgm': 5.93,
   'fga': 13.87,
   'fg3m': 2.93,
   'fg3a': 7.53,
   'ftm': 5.0,
   'fta': 5.8,
   'oreb': 1.0,
   'dreb': 6.4,
  

In [9]:
averages_df = pd.DataFrame(list(chain.from_iterable(averages_list)))
averages_df

,games_played,player_id,season,min,fgm,fga,fg3m,fg3a,ftm,fta,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
0,15,115,2021,32:35,8.87,19.53,5.27,12.87,4.67,4.87,...,6.20,6.47,1.73,0.60,3.27,1.60,27.67,0.454,0.409,0.959
1,15,472,2021,35:28,7.40,17.33,1.27,4.33,3.33,4.87,...,8.73,8.33,1.40,0.20,5.33,3.20,19.40,0.427,0.292,0.685
2,15,192,2021,34:11,5.93,13.87,2.93,7.53,5.00,5.80,...,7.40,8.93,1.13,0.73,4.87,2.33,19.80,0.428,0.389,0.862
3,15,278,2021,34:18,7.40,18.60,2.73,9.07,2.67,3.13,...,4.27,7.67,0.67,0.20,2.47,1.47,20.20,0.398,0.301,0.851
4,6,237,2021,37:03,9.50,20.33,2.83,8.17,3.00,3.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
5,15,140,2021,34:29,10.67,18.80,2.00,4.80,5.60,6.67,...,8.07,5.00,0.53,0.53,3.13,1.20,28.93,0.567,0.417,0.840
6,13,15,2021,32:56,9.46,19.31,1.31,4.69,6.31,9.31,...,11.31,6.00,1.08,1.69,3.00,3.15,26.54,0.490,0.279,0.678
7,14,172,2021,32:56,9.14,20.50,2.86,8.14,3.71,4.14,...,7.71,4.93,2.07,0.43,4.07,2.50,24.86,0.446,0.351,0.897
8,11,79,2021,32:59,8.00,15.18,0.45,1.55,7.18,8.45,...,5.45,5.09,2.09,0.36,1.91,2.00,23.64,0.527,0.294,0.849
9,9,200,2021,31:37,7.78,15.44,1.33,4.00,2.44,2.67,...,8.33,4.33,0.11,0.44,1.89,1.33,19.33,0.504,0.333,0.917


In [10]:
%store -r salary_df


In [11]:
player_id_df = pd.DataFrame(player_id_list)
salary_id_join = player_id_df.join(salary_df)

In [12]:
salary_player_id_df = salary_id_join.rename(columns={0: 'player_id'})
salary_player_id_df

,player_id,Rk,Player,Tm,2021-22
0,115,1,Stephen Curry,GSW,$45780966.00
1,1014,2,John Wall,HOU,$44310840.00
2,472,3,Russell Westbrook,LAL,$44211146.00
3,192,4,James Harden,BRK,$43848000.00
4,278,5,Damian Lillard,POR,$43750000.00
5,237,6,LeBron James,LAL,$41180544.00
6,140,7,Kevin Durant,BRK,$40918900.00
7,15,8,Giannis Antetokounmpo,MIL,$39344970.00
8,172,9,Paul George,LAC,$39344970.00
9,274,10,Kawhi Leonard,LAC,$39344900.00


In [13]:
player_mapper = dict(zip(salary_player_id_df['player_id'].values, salary_player_id_df["Player"].values))
player_mapper
%store player_mapper

Stored 'player_mapper' (dict)


In [14]:
all_data_df = salary_player_id_df.merge(averages_df, on='player_id', how = 'outer')
all_data_df.set_index('Rk').fillna(0)

,player_id,Player,Tm,2021-22,games_played,season,min,fgm,fga,fg3m,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
Rk,,,,,,,,,,,,,,,,,,,,,
1,115,Stephen Curry,GSW,$45780966.00,15.0,2021.0,32:35,8.87,19.53,5.27,...,6.20,6.47,1.73,0.60,3.27,1.60,27.67,0.454,0.409,0.959
2,1014,John Wall,HOU,$44310840.00,0.0,0.0,0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000
3,472,Russell Westbrook,LAL,$44211146.00,15.0,2021.0,35:28,7.40,17.33,1.27,...,8.73,8.33,1.40,0.20,5.33,3.20,19.40,0.427,0.292,0.685
4,192,James Harden,BRK,$43848000.00,15.0,2021.0,34:11,5.93,13.87,2.93,...,7.40,8.93,1.13,0.73,4.87,2.33,19.80,0.428,0.389,0.862
5,278,Damian Lillard,POR,$43750000.00,15.0,2021.0,34:18,7.40,18.60,2.73,...,4.27,7.67,0.67,0.20,2.47,1.47,20.20,0.398,0.301,0.851
6,237,LeBron James,LAL,$41180544.00,6.0,2021.0,37:03,9.50,20.33,2.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
7,140,Kevin Durant,BRK,$40918900.00,15.0,2021.0,34:29,10.67,18.80,2.00,...,8.07,5.00,0.53,0.53,3.13,1.20,28.93,0.567,0.417,0.840
8,15,Giannis Antetokounmpo,MIL,$39344970.00,13.0,2021.0,32:56,9.46,19.31,1.31,...,11.31,6.00,1.08,1.69,3.00,3.15,26.54,0.490,0.279,0.678
9,172,Paul George,LAC,$39344970.00,14.0,2021.0,32:56,9.14,20.50,2.86,...,7.71,4.93,2.07,0.43,4.07,2.50,24.86,0.446,0.351,0.897


In [15]:
# Game Score = Points Scored + (0.4 x Field Goals) – (0.7 x Field Goal Attempts) – (0.4 x (Free Throw Attempts – Free Throws)) + 
# (0.7 x Offensive Rebounds) + (0.3 x Defensive Rebounds) + Steals + (0.7 x Assists) + (0.7 x Blocks) – (0.4 x Personal Fouls) – Turnovers

In [16]:
game_scores_list = []
for player in range (len(all_data_df)):
    a = all_data_df.loc[player, 'pts']
    b = all_data_df.loc[player, 'fgm'] * 0.4
    c = all_data_df.loc[player,'fga'] * 0.7
    d = ((all_data_df.loc[player,'fta']) - (all_data_df.loc[player, 'ftm']) * 0.4)
    e = all_data_df.loc[player,'oreb'] * 0.7
    f = all_data_df.loc[player,'dreb'] * 0.3
    g = all_data_df.loc[player,'stl']
    h = all_data_df.loc[player,'ast'] * 0.7
    i = all_data_df.loc[player,'blk'] * 0.7
    j = all_data_df.loc[player,'pf'] * 0.4
    k = all_data_df.loc[player,'turnover']
    game_score_player = a + b - c - d + e + f + g + h + i - j - k
    game_scores_list.append([game_score_player])
game_scores_list

[[19.442000000000007],
 [nan],
 [10.603],
 [13.373000000000001],
 [12.64],
 [16.328000000000003],
 [19.008000000000003],
 [16.356999999999996],
 [14.802999999999994],
 [nan],
 [nan],
 [16.160999999999998],
 [13.953999999999997],
 [10.566000000000003],
 [16.69],
 [11.486],
 [nan],
 [11.780000000000001],
 [12.018],
 [12.176]]

In [17]:
game_score_df = pd.DataFrame(game_scores_list)
game_score_df.fillna(0)


,0
0,19.442
1,0.000
2,10.603
3,13.373
4,12.640
5,16.328
6,19.008
7,16.357
8,14.803
9,0.000


In [18]:
salaries_df = all_data_df.iloc[: , 4].str.replace('$', '')
salaries_df.astype(float)

/Users/aheeshnagraj/anaconda3/envs/atmdev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


0     45780966.0
1     44310840.0
2     44211146.0
3     43848000.0
4     43750000.0
5     41180544.0
6     40918900.0
7     39344970.0
8     39344970.0
9     39344900.0
10    37980720.0
11    36016200.0
12    36000000.0
13    35500000.0
14    35361360.0
15    35344828.0
16    34916200.0
17    34502130.0
18    31650600.0
19    31610000.0
Name: 2021-22, dtype: float64

In [19]:
salaries_gamescore_nan = pd.merge(salaries_df, game_score_df, right_index=True, left_index=True)
salaries_gamescore_df = salaries_gamescore_nan.fillna(0)
salaries_gamescore_df['2021-22'] = salaries_gamescore_df['2021-22'].astype(float)
salaries_gamescore_df.dtypes

2021-22    float64
0          float64
dtype: object

In [20]:
salaries_gamescore_df = salaries_gamescore_df.rename(columns = {0 : 'game_score'})
salaries_gamescore_df

,2021-22,game_score
0,45780966.0,19.442
1,44310840.0,0.000
2,44211146.0,10.603
3,43848000.0,13.373
4,43750000.0,12.640
5,41180544.0,16.328
6,40918900.0,19.008
7,39344970.0,16.357
8,39344970.0,14.803
9,39344900.0,0.000


In [21]:
# Create the GameScore ratio function and add it to a new dataframe
pd.options.display.float_format = '{:,.2f}'.format
salaries_gamescore_df['GameScore_Salary_Ratio'] = salaries_gamescore_df["2021-22"]/salaries_gamescore_df['game_score']
salaries_gamescore_df = salaries_gamescore_df.replace([np.inf, -np.inf], np.nan).fillna(0)
salaries_gamescore_df

,2021-22,game_score,GameScore_Salary_Ratio
0,"45,780,966.00",19.44,"2,354,745.71"
1,"44,310,840.00",0.00,0.00
2,"44,211,146.00",10.60,"4,169,682.73"
3,"43,848,000.00",13.37,"3,278,845.43"
4,"43,750,000.00",12.64,"3,461,234.18"
5,"41,180,544.00",16.33,"2,522,081.33"
6,"40,918,900.00",19.01,"2,152,719.91"
7,"39,344,970.00",16.36,"2,405,390.35"
8,"39,344,970.00",14.80,"2,657,905.15"
9,"39,344,900.00",0.00,0.00
